In [1]:
import pandas as pd
import numpy as np
import netcomp as nc
import networkx as nx

In [2]:
import scipy.io
mat1 = scipy.io.loadmat('sim/sim1.mat')
mat2 = scipy.io.loadmat('sim/sim2.mat')
mat3 = scipy.io.loadmat('sim/sim3.mat')
mat5 = scipy.io.loadmat('sim/sim5.mat')
mat6 = scipy.io.loadmat('sim/sim6.mat')
mat23 = scipy.io.loadmat('sim/sim23.mat')
mat24 = scipy.io.loadmat('sim/sim24.mat')
mat25 = scipy.io.loadmat('sim/sim15.mat')
mat28 = scipy.io.loadmat('sim/sim18.mat')

In [9]:
i=1
x = mat2['ts'][200*1:200*(1+1),:]
x.shape
x[:, i].shape

(200,)

In [3]:
def patel_tau(d1, d2, npoints = 200, set_bin = False, bin_per = 0):
    XT = npoints
    thre = bin_per

    grotMIN = np.percentile(d1, 10, axis = 0)
    grotMAX = np.percentile(d1, 90, axis = 0)
    tmp = (d1-grotMIN)/(grotMAX-grotMIN)
    tmp2 = np.where(tmp > 1, 1, tmp)
    d1b = np.where(tmp2 < 0, 0, tmp2)

    grotMIN = np.percentile(d2,10)
    grotMAX = np.percentile(d2,90)
    tmp = (d2 - grotMIN)/(grotMAX - grotMIN) 
    tmp2 = np.where(tmp > 1, 1, tmp)
    d2b = np.where(tmp2 < 0, 0, tmp2)
    
    if set_bin == True:
        d1b = np.where(d1b > bin_per, 1, 0)
        d2b = np.where(d2b > bin_per, 1, 0)
        print(f"bin thre is {bin_per}")

    theta1 = np.matmul(np.reshape(d1b, (1, npoints)), d2b)/XT
    theta2 = np.matmul(np.reshape(d1b, (1, npoints)), (1-d2b))/XT
    theta3 = np.matmul(np.reshape(d2b, (1, npoints)), (1-d1b))/XT
    theta4 = np.matmul(np.reshape((1-d1b),(1, npoints)),(1-d2b))/XT
    EEE = np.matmul((theta1+theta2),(theta1+theta3))
    max_theta1 = min(theta1+theta2, theta1+theta3)
    min_theta1 = max(0,2*theta1+theta2+theta3-1);
    
    if theta1 > EEE:
        DDD = 0.5+(theta1-EEE)/(2*(max_theta1-EEE)); 
    else:
        DDD = 0.5-(theta1-EEE)/(2*(EEE-min_theta1));
    
    kappa=(theta1-EEE)/(DDD*(max_theta1-EEE) + (1-DDD)*(EEE-min_theta1))
    #urgh(kappa,'Patel Kappa'); #
    ##p.s. the "urgh"  function just adds a new connection measure value into the database of scores,
    #associated with a method's name
    
    if theta2>theta3: 
        tau_12=1-(theta1+theta3)/(theta1+theta2); 
    else:
        tau_12=(theta1+theta2)/(theta1+theta3)-1;
    #urgh(-tau_12,'Patel Tau');  
    #urgh(-kappa * tau_12,'Tau x Kappa');
    return -kappa * tau_12

In [4]:
def cal_tp_acc(mat1, mat2):
    loc_gt = mat1 == 1
    loc_sim = mat2 == 1
    n_1_gt = len(mat1[loc_gt])
    n_1_sim = len(mat2[loc_sim])
    #cal tp, acc
    n_corr = sum(mat1[loc_gt] == mat2[loc_gt])
    tp = n_corr/n_1_gt
    acc = n_corr/n_1_sim
    return tp, acc

In [5]:
simu_ls = []
for n in range(0, 50):
    x = mat2['ts'][200*n:200*(n+1),:]
    simu = np.empty((10,10,))
    simu[:] = 0
    for i in range(0,10):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 10):
            d2 = x[:, j]
            result = patel_tau(d1, d2, set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat2['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim2 bin75%.csv") 

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__'

In [6]:
simu_ls = []
for n in range(0, 50):
    x = mat3['ts'][200*n:200*(n+1),:]
    simu = np.empty((15,15,))
    simu[:] = 0
    for i in range(0,5):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 5):
            d2 = x[:, j]
            result = patel_tau(d1, d2, set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat3['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim3 bin75%.csv") 

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__'

In [7]:
simu_ls = []
for n in range(0, 50):
    x = mat6['ts'][1200*n:1200*(n+1),:]
    simu = np.empty((10,10,))
    simu[:] = 0
    for i in range(0,10):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 10):
            d2 = x[:, j]
            result = patel_tau(d1, d2, npoints=1200,set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat6['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim6 bin75%.csv") 

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.

bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75
bin thre is 0.75


/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/lliu/.conda/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__'

In [8]:
simu_ls = []
for n in range(0, 50):
    x = mat22['ts'][200*n:200*(n+1),:]
    simu = np.empty((5,5,))
    simu[:] = 0
    for i in range(0,5):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 5):
            d2 = x[:, j]
            result = patel_tau(d1, d2, set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat22['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim22 bin75%.csv") 

NameError: name 'mat22' is not defined

In [ ]:
simu_ls = []
for n in range(0, 50):
    x = mat23['ts'][200*n:200*(n+1),:]
    simu = np.empty((5,5,))
    simu[:] = 0
    for i in range(0,5):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 5):
            d2 = x[:, j]
            result = patel_tau(d1, d2, set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat23['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim23 bin75%.csv") 

In [ ]:
simu_ls = []
for n in range(0, 50):
    x = mat24['ts'][200*n:200*(n+1),:]
    simu = np.empty((5,5,))
    simu[:] = 0
    for i in range(0,5):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 5):
            d2 = x[:, j]
            result = patel_tau(d1, d2, set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat24['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim24 bin75%.csv") 

In [ ]:
simu_ls = []
for n in range(0, 50):
    x = mat25['ts'][100*n:100*(n+1),:]
    simu = np.empty((5,5,))
    simu[:] = 0
    for i in range(0,5):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 5):
            d2 = x[:, j]
            result = patel_tau(d1, d2, npoints=100,set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat25['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim25 bin75%.csv") 

In [ ]:
simu_ls = []
for n in range(0, 50):
    x = mat28['ts'][100*n:100*(n+1),:]
    simu = np.empty((5,5,))
    simu[:] = 0
    for i in range(0,5):
        d1 = x[:, i]
        simu[i, i] = -1
        for j in range(i+1, 5):
            d2 = x[:, j]
            result = patel_tau(d1, d2, npoints= 100,set_bin = True, bin_per = 0.75)
            simu[i, j] = result
    simu_ls.append(simu)

result = []
for i in range(0, 50):
    gt = mat28['net'][i]
    simu = simu_ls[i]
    gt_bin = (gt > 0).astype(np.int_)
    simu_bin = (abs(simu) > 1.5).astype(np.int_)
    tp, acc = cal_tp_acc(gt_bin, simu_bin)
    gt_net = nx.from_numpy_matrix(gt_bin)
    simu_net = nx.from_numpy_matrix(simu_bin)
    ##get adjacent matrix
    a1, a2 = [nx.adjacency_matrix(G) for G in [gt_net, simu_net]] 
    norm_dist = np.linalg.norm(a1 - a2)
    spec_dist = nc.lambda_dist(a1, a2)
    edit = nc.edit_distance(a1, a2)
    netsmile = nc.netsimile(a1, a2)
    result.append([tp, acc, norm_dist, spec_dist, edit, netsmile])
    
result_df = pd.DataFrame(result)
result_df.columns = ['tp', 'acc', 'Euclidean_dist', 'spec_dist', 'edit', 'netsmile']
result_df.to_csv("patel_tau sim28 bin75%.csv") 